In [1]:
from __future__ import division
import numpy as np
from numpy import array
from numpy.random import sample as rs
from numpy import newaxis as na
import pandas as pd 
from scipy.stats import distributions
from scipy.stats import sem
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

from radd.rl import multirace, analyzer, visr
from radd import theta

from scipy.stats import pearsonr
from IPython.display import display, Latex

temporal_dynamics = lambda p, t: np.cosh(p['xb'][:, na] * t)
updateQ = lambda q, winner, r, A: q[winner][-1] + A*(r - q[winner][-1])
boltzmann_choiceP = lambda q, name, B: np.exp(B*q[name][-1])/np.sum([np.exp(B*q[k][-1]) for k in q.keys()])

homedir = os.path.expanduser('~')
cards = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/IGTob/IGTCards.csv"))
cards_scaled=cards.copy()*.01
cards_normed = (cards-cards.mean().mean())/cards.std().mean()

datadf = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/IGTob/IGT_data.csv"))
dat = datadf.dropna()


In [84]:
import ipyparallel as ipp

c = ipp.Client()

c.ids
c[:].apply_sync(lambda : "Hello, World")

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json


IOError: Connection file '~/.ipython/profile_default/security/ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

In [2]:
alpha_hi_dist = distributions.uniform(.25, .10)
alpha_midhi_dist = distributions.uniform(.15, .10)
alpha_midlow_dist = distributions.uniform(.05, .10)
alpha_low_dist = distributions.uniform(.005, .05)

alpha_dist = {'ahi':alpha_hi_dist, 'amidhi':alpha_midhi_dist, 'amidlow':alpha_midlow_dist, 'alow': alpha_low_dist}

In [3]:
alphas_go=hs([alpha_dist[k].rvs(10) for k in alpha_dist.keys()]) 
alphas_no=hs([alpha_dist[k].rvs(10) for k in alpha_dist.keys()]) 

np.random.shuffle(alphas_go)
np.random.shuffle(alphas_no)

betas = np.linspace(.05, 8, 10)

df = pd.DataFrame({'g':alphas_go, 'n':alphas_no, 'd': alphas_go-alphas_no}, index=np.arange(len(alphas_go)))
dfsorted = df.sort_values(by='d')

ago = dfsorted['g'].values
ano = dfsorted['n'].values
plt.plot(ago-ano)

# Simulate effects of $\alpha, \beta$

In [4]:
p={'vd':.7, 'vi':.4, 'a':.4, 'tr':.2, 'xb': 1.5}

trialsdf, igtdf = multirace.run_full_sims(p, env=cards_scaled.copy(), alphas_go=ago, alphas_no=ano, betas=betas, nagents=50)

In [53]:
igtdf
igtdf.loc[:, 'adiff'] = igtdf.a_go - igtdf.a_no

In [82]:
data = igtdf.pivot_table('P', 'adiff', 'bgroup')

In [77]:
plt.plot(igtdf.groupby(''.adiff, 

,group,agent,bgroup,agroup,a_go,a_no,beta,P,Q,adiff
0,0.0,24.5,0.0,0.0,0.04234,0.34662,0.05,2.92,-1.76,-0.30428
1,1.0,24.5,0.0,1.0,0.10036,0.31964,0.05,1.04,-2.20,-0.21928
2,2.0,24.5,0.0,2.0,0.03124,0.24278,0.05,3.12,-1.76,-0.21153
3,3.0,24.5,0.0,3.0,0.07655,0.27482,0.05,2.88,-1.16,-0.19827
4,4.0,24.5,0.0,4.0,0.13775,0.33511,0.05,4.36,-0.28,-0.19736
5,5.0,24.5,0.0,5.0,0.05420,0.25067,0.05,2.88,0.72,-0.19647
6,6.0,24.5,0.0,6.0,0.01075,0.20321,0.05,0.44,-1.80,-0.19245
7,7.0,24.5,0.0,7.0,0.04951,0.22980,0.05,-1.56,-1.00,-0.18030
8,8.0,24.5,0.0,8.0,0.04133,0.20922,0.05,0.68,0.80,-0.16790
9,9.0,24.5,0.0,9.0,0.08194,0.23611,0.05,-1.56,-1.00,-0.15416


In [79]:
[bg.groupby('agroup').mean().P.plot(label=b) for b, bg in igtdf.groupby('bgroup')]
plt.legend()

In [47]:
trialsdf.head()
ab_pivot = trialsdf.pivot_table('vdiff', 'adiff', 'bgroup')
data = ab_pivot.reset_index(drop=True)

In [50]:
ab_pivot.size

400

In [ ]:
igtdf

In [30]:
trialsdf.groupby('trial').mean()

(100, 14)

In [46]:
tdf.loc[:, 'adiff'] = tdf.a_go - tdf.a_no

In [ ]:
tdf.

In [83]:
import plotly.plotly as plotly
import plotly.graph_objs as go
import plotly as py
py.offline.init_notebook_mode() # run at the start of every notebook
datam = [go.Surface(z=data.as_matrix())]
plotly.plot(go.Figure(data=datam))

u'https://plot.ly/~dunovank/86'

In [20]:
example_data.

(25, 25)

# Single Sim

In [ ]:
p={'vd':.7, 'vi':.4, 'a':.4, 'tr':.2, 'xb': 1.5}

pdict = theta.random_inits(['vd', 'vi', 'tr', 'a', 'xb'])

p = deepcopy(pdict)
output = multirace.run_trials(p, cards_scaled, nblocks=2, si=.01, a_go=.4, a_no=.01, beta=8)
choices, rts, all_traces, qdict, qdict_go, qdict_no, choice_prob, vdhist, vihist = output
Pi,Qi = analyzer.igt_scores(np.asarray(choices))